In [0]:
article1 = {
    "title": "How to Request a New Article Topic Be Written on wikiHow",
    "url": "https://www.wikihow.com/Request-a-New-Article-Topic-Be-Written-on-wikiHow",
    "title_description": "At wikiHow, you can suggest a topic that has not yet been covered. This article will tell you how.",
    "category_hierarchy": [],
    "method/part": [],
    "steps": [
        [
            "Review how to refrain from making unhelpful Suggested Topics on wikiHow",
            "\nDo not post a suggestion more than once.\nDo not post a suggestion too similar to an existing article title.\nA topic that either makes no sense or violates wikiHow's rules will be deleted."
        ],
        [
            "Search wikiHow",
            " Duplicate suggestions will eventually be merged or deleted."
        ],
        [
            "Click on the \"Help Us\" button in the top right corner of any wikiHow page",
            " Then click on the Request a New Article link in the drop-down box that opens."
        ],
        [
            "Enter a title according to the Title policy",
            "\nBe clear, concise and specific.\nWith the exception of small prepositions (the, a, an, etc.), coordinate conjunctions, and preposition (except for when used as part of a verb), all other words should contain at least one capital letter at the beginning.\nDo not type \"How to\" as this is automatically added.\nDon't use possessive words. Words such as mine, my, ours, his, hers, etc can make the writer think he is writing the article just for you. wikiHow articles are for everyone and should be specific in description, but not specific to certain people.\nUse proper capitalization for products, companies, location, etc. Examples are iPhone, PC, BBC, USA, and Myspace.\nStart your request with an action verb, such as \"Grow\", \"Make\", \"Answer\", etc."
        ],
        [
            "Categorize your suggestion to help others locate your topic",
            "\nChoose a category from the drop down menu. For example, recipes belong in the \"Food and Entertaining\" category and any fashion would be under the \"Personal Care and Style\" category.\nIf a category does not fit your suggestion, choose the \"Other\" category."
        ],
        [
            "Determine if you want to be emailed when your article is written",
            "\nIf you want wikiHow to email you, select the box labelled \"E-mail me when my how-to is written:\"\nIf your settings allow editors to contact you via email, your address that was applied, will automatically be used and automatically typed into the box upon entry."
        ],
        [
            "Look at the reCAPTCHA image",
            " Type it into the box directly below the image."
        ],
        [
            "Click the \"Submit\" button, when you are finished",
            ""
        ],
        [
            "Be patient",
            " Suggestions may take a while to be accepted by those users who have certain rights and time to be finally answered."
        ]
    ]
}

In [3]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [0]:
import nltk, string

from nltk import RegexpParser
from nltk.tree import Tree

def is_imperative(tagged_sent):
    # if the sentence is not a question...
    if tagged_sent[-1][0] != "?":
        # catches simple imperatives, e.g. "Open the pod bay doors, HAL!"
        if tagged_sent[0][1] == "VB" or tagged_sent[0][1] == "MD":
            return True

        # catches imperative sentences starting with words like 'please', 'you',...
        # E.g. "Dave, stop.", "Just take a stress pill and think things over."
        else:
            chunk = get_chunks(tagged_sent)
            # check if the first chunk of the sentence is a VB-Phrase
            if type(chunk[0]) is Tree and chunk[0].label() == "VB-Phrase":
                return True

    # Questions can be imperatives too, let's check if this one is
    else:
        # check if sentence contains the word 'please'
        pls = len([w for w in tagged_sent if w[0].lower() == "please"]) > 0
        # catches requests disguised as questions
        # e.g. "Open the doors, HAL, please?"
        chunk = get_chunks(tagged_sent)
        if pls and (tagged_sent[0][1] == "VB" or tagged_sent[0][1] == "MD"):
            return True

        # catches imperatives ending with a Question tag
        # and starting with a verb in base form, e.g. "Stop it, will you?"
        elif type(chunk[-1]) is Tree and chunk[-1].label() == "Q-Tag":
            if (chunk[0][1] == "VB" or
                (type(chunk[0]) is Tree and chunk[0].label() == "VB-Phrase")):
                return True

    return False

# chunks the sentence into grammatical phrases based on its POS-tags
def get_chunks(tagged_sent):
    chunkgram = r"""VB-Phrase: {<DT><,>*<VB>}
                    VB-Phrase: {<RB><VB>}
                    VB-Phrase: {<UH><,>*<VB>}
                    VB-Phrase: {<UH><,><VBP>}
                    VB-Phrase: {<PRP><VB>}
                    VB-Phrase: {<NN.?>+<,>*<VB>}
                    Q-Tag: {<,><MD><RB>*<PRP><.>*}"""
    chunkparser = RegexpParser(chunkgram)
    return chunkparser.parse(tagged_sent)

# Tokenizer that I implemented from CIS421
def tokenize(text):
    tokens = []
    words = text.split(' ')
    for word in words:
        if len(word) == 0:
            continue
        w = []
        for c in word:
            if c in string.punctuation:
                if len(w) != 0:
                    tokens.append(''.join(w))
                    w = []
                tokens.append(c)
            else:
                if c not in string.whitespace:
                    w.append(c)
        if len(w) != 0:
            tokens.append(''.join(w))
    return tokens

# text = 'Wash your hands frequently.'
# tags = nltk.pos_tag(tokenize(text))
# print(tags)
# print(is_imperative(tags))


In [0]:
# refered from https://stackoverflow.com/questions/41051125/turning-a-sentence-from-first-to-second-person

forms = {"am" : "are", "are" : "am", 'i' : 'you', 'my' : 'your', 'me' : 'you', 'mine' : 'yours', 'you' : 'I', 
'your' : 'my', 'yours' : 'mine'}

def translate(text):
  # print(text)
  result = text
  for word in text.split():
    # print(word)
    if word.lower() in forms: 
      result = result.replace(word, forms[word.lower()])
      # print(result)
  # print(result)
  return result

def changePerson(text):
  result = ""
  tags = nltk.pos_tag(tokenize(text))
  isImp = is_imperative(tags)

  if isImp:
    result = "You " + text
  else:
    result = translate(text)
  return result


ajzj


'Wash your hands'

In [51]:
def identifyObjects(text):
  tags = nltk.pos_tag(tokenize(text))
  objects = []
  for t in tags:
    if t[1]=="NN":
      objects.append(t[0])
  return objects
# identifyObjects("You have a dog and a harness")

['dog', 'harness']